# Project
- Work solo or with one other person
- Topic should be something interesting and fun for you
- Your code should be written in Python 3.6
- Effort roughly equivalent to two homeworks
    - perhaps 150-200 lines of code
    - sometimes people really get into it, which is fine
        - have seen 1,000 line programs, but this is NOT necessary
- All I'm really looking for is to see you've learned some Python and can do something with it
- Happy to discuss your project with you if you have questions
- Hangman games are discouraged(seen a pile of them)

# If you did a solo project, fill in this cell

```
name:
UNI: 
```

Submit a zip file named 'yourUNI.zip' to CW2 that includes this notebook, your source, documentation, and data files. The zip must expand into a directory named 'YourUNI', and all your files should be under the YourUNI directory.


# If you did a project with a partner, fill in this cell

```
name1:
UNI1:

name2:
UNI2:

roughly, who did what:

```

Each of you should submit the SAME zip file named   
'UNI1-UNI2.zip' to CW2. The zip should include this notebook, your source, documentation, and data files. The zip must expand into a directory named 'UNI1-UNI2', and all your files should be under the UNI1-UNI2 directory.


# What does your project do?

...



# What modules did you use, if any?

......

# Explain how we can run your project
- Make sure you include all python and data files
- If your project has graphics, include some snapshots
- Any documentation should be written in a notebook
    - do NOT submit ASCII/plain text, PDFs, Word files, etc.
- If for some reason it is not feasible for us to run your project, make an appointment with Larry to demo it


.....


# An evaluation of Python's suitability to your task
    - Were any problems encountered?
    - Was Python a good fit for your project?
        - In retrospect, would you have preferred to use another language for your project - R, Java, Matlab, etc
        - Be honest! You don't have to praise Python. I want to hear about your actual experience. 
        

.....

In [4]:
from selenium import webdriver
import requests
import time

def save_page(url, out_file, wait_for_class='search-card'):
    """ Saves a page, using the Selenium browser testing framework to run any Javascript
        , writing the resulting final DOM as HTML source to disk.
        Waits until element of a given class is loaded to make sure Javascript has actually
        completed before source is retrieved. """

    browser = webdriver.Chrome()
    browser.get(url)
    # Wait until elment with given class is loaded
    while True:
        try:
            # if the element cannot be found, this will throw
            browser.find_element_by_class_name(wait_for_class)
            break # otherwise the loop will end here
        except:
            pass # do nothing, try again
    # source has loaded - write out result
    open(out_file, 'w').write(browser.page_source)

#loop through 42 pages of search results & save to working directory
page = 0    
while page <= 41:
    url = 'https://www.vitals.com/search?display_type=Doctor&page=' + str(page)
    page += 1
    save_page(url, '/home/katya/Documents/Python/Final Project/Pages/Vitals' + str(page) + ".html")
   

In [14]:
from bs4 import BeautifulSoup

def parse_page(filepath):
    page_contents = open(filepath).read().strip()
    my_soup = BeautifulSoup(page_contents, "html.parser")
    for card in my_soup.findAll("div", attrs = {"class": "search-card"}):
        name_element = card.find("span", attrs = {"class": "company"})
        if name_element == None:
            continue
        name = name_element.text.strip()
        
        
        print(name)
    
for page in range(1, 43):
    parse_page("/home/katya/Documents/Python/Final Project/Pages/Vitals" + str(page) + ".html")


Looking Good Cosmetic Surgery
Metropolitan Vein and Aesthetic Center
Angel Caraballo, MD
Maria D Dominicis
Dr. Ajay K Agarwala
Dr. Paul J Frank
Dr. Saima Sultan
Dr. Kumud B Jindal
Dr. Cyrus A Boquin
Dr. Maria Del Carmen D Hidalgo
Dr. Dalsia J Acosta
Dr. Osiris H Rivas
Dr. Eduard Fuzaylov
Dr. Gopal Narasimhan
Dr. Jin M Choi
Dr. Suzanne Friedman
Dr. Melissa E Glassman
Dr. Juan S Lopez
Dr. Juan S Lopez
Dr. Sibdas M Mukhopadhyay
Arsenio Miguel Tio MD
Dr. Miguel A Andujar
Andrea M Valdez
Dr. Sarah A Guzman-Marte
Dr. Gozen Tuysuzoglu
Dr. Gozen Tuysuzoglu
Jacqueline Carreno
Dr. Douglas E Hertford
Dr. Douglas E Hertford
Dr. Michael D Winn
Dr. Michael D Winn
Dr. Sarmistha B Mukherjee
Dr. John S Walsh
Dr. Irene G Cergnul
Dr. Elizabeth M Lutas-Lutas
Dr. Ruben U Carvajal
Dr. Edwin L Naamon
Dr. Claudette O Tiru
NEW YORK PRESBYTERIAN ALLEN HOSPITAL
Aziz M Ahmad
Ashwin Vasan
Dr. Donald M Kim
Dr. Mary J Fink
Richard E Glass
Dr. Nydia E Providence
Dr. Jason B Diamond
Dr. Smriti H Deshmukh
Thuy-Tien L D

Dr. Karen L Hennessey
Dr. Anand R Kapadia
Dr. Charudatta Wankhade
Puneet M Bedi
Dr. Anthony V Carella
Dr. Rajasekhar V Addepalli
Dr. Rajasekhar V Addepalli
Dr. Shefali Khanna
Dr. Angelica M Patino Gonzalez
Dr. Amir Hassan Navaei
Dr. Lindsay A Flax
Dr. Sevak Keshishyan
Dr. Samantha M Arevalo Marcano
Dr. Maria C Srinivasan
Dr. Shauna Seaman
Dr. Bharat Kothakota
Dr. Evelyn N Erickson
Dr. Firas Rabbat
Dr. Michelle J Vaz
Dr. Nail Cemalovic
Dr. Samuel T Gatzert
Dr. Isabel C Gomez
Erkan Ozturk
Dr. Joe T Lequerica
Dr. Minakshi Ramchand
Dr. Riva Z Robinson
Dr. Elaine B Josephson
Dr. Cynthia I Victor
Abraham Hamaoui
Dr. Adriana B Dager
Dr. Riva Z Robinson
Dr. Elaine B Josephson
Dr. Cynthia I Victor
Abraham Hamaoui
Dr. Adriana B Dager
Dr. Jessica L Betancourt
Robert J Terdiman
Dr. Diana King
Dr. Zeev Stegman
Dr. Shan D Babeendran
Dr. Kara J Suche
Dr. Ohannes A Nercessian
Dr. Paul Jendrek
Dr. Victor J Bucalo
Dr. Daniel L Labovitz
Dr. Apolinario B Mateo Jr
Dr. Lori A Ciuffo
Dr. Calvin H Hwang
Dr. E